In [1]:
import os 
import math
import random
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from numpy import count_nonzero
dir_ = '../../../../../data'
group_dir_ = os.path.join(dir_, 'groups/high')

In [2]:
file_name = 'normalized_to_rating_filter_track_5_user_100.csv'
df = pd.read_pickle(os.path.join(dir_, file_name[:-3] + 'pkl'))
svd = pd.read_pickle(os.path.join(dir_, 'prediction_svd_top_N_' + file_name[:-3] + 'pkl'))
test = pd.read_pickle(os.path.join(dir_, 'test_' + file_name[:-3] + 'pkl'))
test.sort_values(by=['uid','tid'])

num_user = len(svd['uid'].unique())
num_user

953

In [3]:
pop_dict = {}
for i in tqdm(df['tid'].unique()):
    pop_dict[i] = len(df[df['tid']==i])/num_user

In [4]:
svd['count'] = 1
svd[:5]

,uid,tid,rating,count
0,0,9,2.243909,1
1,0,15,2.160610,1
2,0,22,2.145235,1
3,0,28,2.204445,1
4,0,33,2.166677,1


In [5]:
# Count POP to DF
tid_list = []
pop_list = []
for i in df['tid'].unique():
    tid_list.append(i)
    pop_list.append(len(df[df['tid']==i])/num_user*5)
    
d = {'tid': tid_list, 'rating': pop_list}
df_pop = pd.DataFrame(data=d)
df_pop = df_pop.sort_values(by=['rating'], ascending=False)
df_pop[:10]

,tid,rating
70,70,4.034627
5521,5521,2.911857
390,390,2.822665
2519,2519,2.801679
83,83,2.796432
13496,13496,2.785939
210,210,2.686254
5716,5716,2.681007
1036,1036,2.544596
1464,1464,2.528856


In [6]:
lambdas = []
for i in range(11):
    lambdas.append(i/10)
lambdas

[0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]

In [7]:
top_n_size = 20

In [8]:
groups_n =[]
group_sizes = ['2', '3', '4', '5', '6', '7', '8', '9', '10']
for i in range(len(group_sizes)):
    groups = np.loadtxt(os.path.join(group_dir_, group_sizes[i] + '.csv'), delimiter=',')
    groups = groups[int(len(groups)/5*0):int(len(groups)/5*1)]
    groups_n.append(groups)

NOVs = []
COVs = []
HitNOVs = []
HitCOVs = []
for lambda_ in tqdm(lambdas):
    top_n_lists_g = []
    top_n_items_g = []
    for groups in groups_n:

        group_size = len(groups[0])
        top_n_lists = []
        top_n_items = pd.DataFrame()

        for group in groups:
            rating_table = pd.DataFrame() 
            for member in group:
                prediction = svd[svd['uid'] == member].copy()
                if len(rating_table) == 0:
                    rating_table = prediction
                else:
                    rating_table = rating_table.set_index('tid').add(prediction.set_index('tid'), fill_value=0).reset_index()
            rating_table['rating'] /= group_size 
            rating_table['rating'] *= lambda_
            pop = df_pop.copy()
            pop['rating'] *= (1 - lambda_)
            rating_table = rating_table.set_index('tid').add(pop.set_index('tid'), fill_value=0).reset_index()
    #         rating_table = rating_table[rating_table['count'] == group_size]
            rating_table = rating_table.sort_values(by=['rating'],  ascending=False)
            rating_table = rating_table[:top_n_size]
            
            top_n_list = []
            for _, row in rating_table.iterrows():
                top_n_list.append(row[0])
            top_n_lists.append(top_n_list)
            
            if len(top_n_items) == 0:
                top_n_items = rating_table
            else:
                top_n_items = top_n_items.set_index('tid').add(rating_table.set_index('tid'), fill_value=0).reset_index()
                
        top_n_lists_g.append(top_n_lists)   
        top_n_items_g.append(top_n_items)
        
    COV = []    
    NOV = []
    HitNOV = []
    HitCOV = []
    for i in range(len(groups_n)):
        groups = groups_n[i]
        top_n_lists = top_n_lists_g[i]
        top_n_items = top_n_items_g[i]

        
        nov = 0
        hitNov = 0
        hits = np.zeros(len(df['tid'].unique()), dtype=bool)
        for j in (range(len(groups))):

            group = groups[j]
            top_n_list = top_n_lists[j]
            
            for k in range(len(group)):
                for l in range(top_n_size):
                    uid = group[k]
                    tid = top_n_list[l]
                    t = test[test['uid'] == uid]
                    t = t[t['tid'] == tid]
                    nov += (1-pop_dict[tid])
                    if len(t) > 0:
                        hits[int(tid)] = True
                        hitNov += (1-pop_dict[tid])
        NOV.append(nov/top_n_size/(len(groups)*len(groups[0])))
        COV.append(len(top_n_items)/len(df['tid'].unique()))
        HitNOV.append(hitNov/top_n_size/(len(groups)*len(groups[0])))
        HitCOV.append(list(hits).count(True)/len(df['tid'].unique()))
    NOVs.append(NOV)
    COVs.append(COV)
    HitNOVs.append(HitNOV)
    HitCOVs.append(HitCOV)
    
# for NOV in NOVs:
#     print(NOV)
# print()
for HitNOV in HitNOVs:
    print(HitNOV)
print()  
# for COV in COVs:
#     print(COV)
print()    
for HitCOV in HitCOVs:
    print(HitCOV)


[0.05320333449924224, 0.06171589530634876, 0.06643406785589372, 0.06911957227801936, 0.07193019849597769, 0.07585646253012461, 0.07750918153200421, 0.07763009599316004, 0.07187902863138966]
[0.0699548534193516, 0.07362007248479986, 0.07903812521860794, 0.07855743766551751, 0.08110080010493187, 0.08309445014586672, 0.08438221406086044, 0.08408663998030912, 0.07580947384200278]
[0.08530631015765681, 0.08632202897903606, 0.09388990905911158, 0.09029930545145667, 0.0880120890171389, 0.09086343876480289, 0.0878934942287513, 0.09182892231160865, 0.07958776795083203]
[0.1062405269907892, 0.10676363555178507, 0.10708683105981108, 0.09974666466796593, 0.10028200419727179, 0.10367895483320455, 0.0929348111227701, 0.1047381239231537, 0.08656573227402198]
[0.12681525526926007, 0.1291803412888005, 0.12816850297306737, 0.12229950532154105, 0.11476040573627143, 0.11905146270308915, 0.10762001573976905, 0.11593408728770742, 0.09503672612801685]
[0.14634552355783545, 0.1515423485372804, 0.156750612102

In [9]:
groups_n =[]
group_sizes = ['2', '3', '4', '5', '6', '7', '8', '9', '10']
for i in range(len(group_sizes)):
    groups = np.loadtxt(os.path.join(group_dir_, group_sizes[i] + '.csv'), delimiter=',')
    groups = groups[int(len(groups)/5*1):int(len(groups)/5*2)]
    groups_n.append(groups)

NOVs = []
COVs = []
HitNOVs = []
HitCOVs = []
for lambda_ in tqdm(lambdas):
    top_n_lists_g = []
    top_n_items_g = []
    for groups in groups_n:

        group_size = len(groups[0])
        top_n_lists = []
        top_n_items = pd.DataFrame()

        for group in groups:
            rating_table = pd.DataFrame() 
            for member in group:
                prediction = svd[svd['uid'] == member].copy()
                if len(rating_table) == 0:
                    rating_table = prediction
                else:
                    rating_table = rating_table.set_index('tid').add(prediction.set_index('tid'), fill_value=0).reset_index()
            rating_table['rating'] /= group_size 
            rating_table['rating'] *= lambda_
            pop = df_pop.copy()
            pop['rating'] *= (1 - lambda_)
            rating_table = rating_table.set_index('tid').add(pop.set_index('tid'), fill_value=0).reset_index()
    #         rating_table = rating_table[rating_table['count'] == group_size]
            rating_table = rating_table.sort_values(by=['rating'],  ascending=False)
            rating_table = rating_table[:top_n_size]
            
            top_n_list = []
            for _, row in rating_table.iterrows():
                top_n_list.append(row[0])
            top_n_lists.append(top_n_list)
            
            if len(top_n_items) == 0:
                top_n_items = rating_table
            else:
                top_n_items = top_n_items.set_index('tid').add(rating_table.set_index('tid'), fill_value=0).reset_index()
                
        top_n_lists_g.append(top_n_lists)   
        top_n_items_g.append(top_n_items)
        
    COV = []    
    NOV = []
    HitNOV = []
    HitCOV = []
    for i in range(len(groups_n)):
        groups = groups_n[i]
        top_n_lists = top_n_lists_g[i]
        top_n_items = top_n_items_g[i]

        
        nov = 0
        hitNov = 0
        hits = np.zeros(len(df['tid'].unique()), dtype=bool)
        for j in (range(len(groups))):

            group = groups[j]
            top_n_list = top_n_lists[j]
            
            for k in range(len(group)):
                for l in range(top_n_size):
                    uid = group[k]
                    tid = top_n_list[l]
                    t = test[test['uid'] == uid]
                    t = t[t['tid'] == tid]
                    nov += (1-pop_dict[tid])
                    if len(t) > 0:
                        hits[int(tid)] = True
                        hitNov += (1-pop_dict[tid])
        NOV.append(nov/top_n_size/(len(groups)*len(groups[0])))
        COV.append(len(top_n_items)/len(df['tid'].unique()))
        HitNOV.append(hitNov/top_n_size/(len(groups)*len(groups[0])))
        HitCOV.append(list(hits).count(True)/len(df['tid'].unique()))
    NOVs.append(NOV)
    COVs.append(COV)
    HitNOVs.append(HitNOV)
    HitCOVs.append(HitCOV)
    
# for NOV in NOVs:
#     print(NOV)
# print()
for HitNOV in HitNOVs:
    print(HitNOV)
print()  
# for COV in COVs:
#     print(COV)
print()    
for HitCOV in HitCOVs:
    print(HitCOV)


[0.048677988936820765, 0.054977190358182146, 0.06397560335781749, 0.06408900124010312, 0.0637220181881777, 0.06431050586349644, 0.06003469903653536, 0.06869275711528247, 0.05810991605456457]
[0.05770698120295893, 0.06473324254128444, 0.07227570828961186, 0.06936659353238583, 0.07258328961175241, 0.07156924921877707, 0.06350996852046173, 0.07176946096148613, 0.06021970094438619]
[0.06488250229943139, 0.07338899927812889, 0.0786835431969221, 0.07835686349327495, 0.07763149265477444, 0.07568349803397025, 0.06723146999904611, 0.0783387094684751, 0.06543743441762859]
[0.07648102807249367, 0.0867733845340955, 0.08914130814970281, 0.08402174949918921, 0.08666382476390351, 0.08031664033762666, 0.07335149766288276, 0.08371095824750949, 0.068262722980063]
[0.081557265555167, 0.10043349482410913, 0.09753891220706559, 0.09152294190594298, 0.09328163256383352, 0.08942379760847755, 0.0839400696365544, 0.08644760535281701, 0.07441500524658975]
[0.08878395709455524, 0.10669926250065125, 0.09931750612

In [10]:
groups_n =[]
group_sizes = ['2', '3', '4', '5', '6', '7', '8', '9', '10']
for i in range(len(group_sizes)):
    groups = np.loadtxt(os.path.join(group_dir_, group_sizes[i] + '.csv'), delimiter=',')
    groups = groups[int(len(groups)/5*2):int(len(groups)/5*3)]
    groups_n.append(groups)

NOVs = []
COVs = []
HitNOVs = []
HitCOVs = []
for lambda_ in tqdm(lambdas):
    top_n_lists_g = []
    top_n_items_g = []
    for groups in groups_n:

        group_size = len(groups[0])
        top_n_lists = []
        top_n_items = pd.DataFrame()

        for group in groups:
            rating_table = pd.DataFrame() 
            for member in group:
                prediction = svd[svd['uid'] == member].copy()
                if len(rating_table) == 0:
                    rating_table = prediction
                else:
                    rating_table = rating_table.set_index('tid').add(prediction.set_index('tid'), fill_value=0).reset_index()
            rating_table['rating'] /= group_size 
            rating_table['rating'] *= lambda_
            pop = df_pop.copy()
            pop['rating'] *= (1 - lambda_)
            rating_table = rating_table.set_index('tid').add(pop.set_index('tid'), fill_value=0).reset_index()
    #         rating_table = rating_table[rating_table['count'] == group_size]
            rating_table = rating_table.sort_values(by=['rating'],  ascending=False)
            rating_table = rating_table[:top_n_size]
            
            top_n_list = []
            for _, row in rating_table.iterrows():
                top_n_list.append(row[0])
            top_n_lists.append(top_n_list)
            
            if len(top_n_items) == 0:
                top_n_items = rating_table
            else:
                top_n_items = top_n_items.set_index('tid').add(rating_table.set_index('tid'), fill_value=0).reset_index()
                
        top_n_lists_g.append(top_n_lists)   
        top_n_items_g.append(top_n_items)
        
    COV = []    
    NOV = []
    HitNOV = []
    HitCOV = []
    for i in range(len(groups_n)):
        groups = groups_n[i]
        top_n_lists = top_n_lists_g[i]
        top_n_items = top_n_items_g[i]

        
        nov = 0
        hitNov = 0
        hits = np.zeros(len(df['tid'].unique()), dtype=bool)
        for j in (range(len(groups))):

            group = groups[j]
            top_n_list = top_n_lists[j]
            
            for k in range(len(group)):
                for l in range(top_n_size):
                    uid = group[k]
                    tid = top_n_list[l]
                    t = test[test['uid'] == uid]
                    t = t[t['tid'] == tid]
                    nov += (1-pop_dict[tid])
                    if len(t) > 0:
                        hits[int(tid)] = True
                        hitNov += (1-pop_dict[tid])
        NOV.append(nov/top_n_size/(len(groups)*len(groups[0])))
        COV.append(len(top_n_items)/len(df['tid'].unique()))
        HitNOV.append(hitNov/top_n_size/(len(groups)*len(groups[0])))
        HitCOV.append(list(hits).count(True)/len(df['tid'].unique()))
    NOVs.append(NOV)
    COVs.append(COV)
    HitNOVs.append(HitNOV)
    HitCOVs.append(HitCOV)
    
# for NOV in NOVs:
#     print(NOV)
# print()
for HitNOV in HitNOVs:
    print(HitNOV)
print()  
# for COV in COVs:
#     print(COV)
print()    
for HitCOV in HitCOVs:
    print(HitCOV)


[0.04838475417807697, 0.05355966538416704, 0.056117086393844, 0.06041423075001252, 0.058903257000596676, 0.06487321702431886, 0.06731636935991606, 0.05707382793776643, 0.064340241343127]
[0.05327783891689919, 0.05991168240643588, 0.06077081147254284, 0.0638929695697797, 0.061928790403884555, 0.06824890744093269, 0.07180876180482686, 0.06115515655564627, 0.06848439139559288]
[0.06291300898318535, 0.06742742217558587, 0.06650533403287862, 0.06790386248938189, 0.06648252165504194, 0.07098289957681357, 0.07693467995802726, 0.06844143899058207, 0.07221209338929699]
[0.06986442044378473, 0.07493186720298473, 0.07246502273522208, 0.07545645330535151, 0.07085879472460556, 0.07713409360838536, 0.08263641133263379, 0.07152656328941742, 0.07465569254984265]
[0.07187731937655165, 0.0818799551125102, 0.08189882826162988, 0.08764153300354767, 0.07888607699113233, 0.08458828684432047, 0.0886988457502623, 0.07588706229839491, 0.07855849947534108]
[0.07599589230414867, 0.08502936632855314, 0.083871983

In [11]:
groups_n =[]
group_sizes = ['2', '3', '4', '5', '6', '7', '8', '9', '10']
for i in range(len(group_sizes)):
    groups = np.loadtxt(os.path.join(group_dir_, group_sizes[i] + '.csv'), delimiter=',')
    groups = groups[int(len(groups)/5*3):int(len(groups)/5*4)]
    groups_n.append(groups)

NOVs = []
COVs = []
HitNOVs = []
HitCOVs = []
for lambda_ in tqdm(lambdas):
    top_n_lists_g = []
    top_n_items_g = []
    for groups in groups_n:

        group_size = len(groups[0])
        top_n_lists = []
        top_n_items = pd.DataFrame()

        for group in groups:
            rating_table = pd.DataFrame() 
            for member in group:
                prediction = svd[svd['uid'] == member].copy()
                if len(rating_table) == 0:
                    rating_table = prediction
                else:
                    rating_table = rating_table.set_index('tid').add(prediction.set_index('tid'), fill_value=0).reset_index()
            rating_table['rating'] /= group_size 
            rating_table['rating'] *= lambda_
            pop = df_pop.copy()
            pop['rating'] *= (1 - lambda_)
            rating_table = rating_table.set_index('tid').add(pop.set_index('tid'), fill_value=0).reset_index()
    #         rating_table = rating_table[rating_table['count'] == group_size]
            rating_table = rating_table.sort_values(by=['rating'],  ascending=False)
            rating_table = rating_table[:top_n_size]
            
            top_n_list = []
            for _, row in rating_table.iterrows():
                top_n_list.append(row[0])
            top_n_lists.append(top_n_list)
            
            if len(top_n_items) == 0:
                top_n_items = rating_table
            else:
                top_n_items = top_n_items.set_index('tid').add(rating_table.set_index('tid'), fill_value=0).reset_index()
                
        top_n_lists_g.append(top_n_lists)   
        top_n_items_g.append(top_n_items)
        
    COV = []    
    NOV = []
    HitNOV = []
    HitCOV = []
    for i in range(len(groups_n)):
        groups = groups_n[i]
        top_n_lists = top_n_lists_g[i]
        top_n_items = top_n_items_g[i]

        
        nov = 0
        hitNov = 0
        hits = np.zeros(len(df['tid'].unique()), dtype=bool)
        for j in (range(len(groups))):

            group = groups[j]
            top_n_list = top_n_lists[j]
            
            for k in range(len(group)):
                for l in range(top_n_size):
                    uid = group[k]
                    tid = top_n_list[l]
                    t = test[test['uid'] == uid]
                    t = t[t['tid'] == tid]
                    nov += (1-pop_dict[tid])
                    if len(t) > 0:
                        hits[int(tid)] = True
                        hitNov += (1-pop_dict[tid])
        NOV.append(nov/top_n_size/(len(groups)*len(groups[0])))
        COV.append(len(top_n_items)/len(df['tid'].unique()))
        HitNOV.append(hitNov/top_n_size/(len(groups)*len(groups[0])))
        HitCOV.append(list(hits).count(True)/len(df['tid'].unique()))
    NOVs.append(NOV)
    COVs.append(COV)
    HitNOVs.append(HitNOV)
    HitCOVs.append(HitCOV)
    
# for NOV in NOVs:
#     print(NOV)
# print()
for HitNOV in HitNOVs:
    print(HitNOV)
print()  
# for COV in COVs:
#     print(COV)
print()    
for HitCOV in HitCOVs:
    print(HitCOV)


[0.0529358231964039, 0.06079830025377127, 0.05911638684854851, 0.060691118954497796, 0.05998600909408885, 0.060551987362060855, 0.060455022417246984, 0.06569896234114489, 0.07766789087093387]
[0.05889588434184459, 0.06497622290192231, 0.06168153200419732, 0.0650100162167319, 0.06038223592165093, 0.06330096975427514, 0.06327446818658782, 0.06944412058088167, 0.08115031479538296]
[0.06619738836423002, 0.07142766776063648, 0.06427247289261978, 0.07143613469426692, 0.0642526888772298, 0.06880931240847298, 0.067988052084327, 0.07159975645460076, 0.08272822665267571]
[0.07296289819025045, 0.08075803918942057, 0.07080535152151105, 0.07413669751025473, 0.07111041885274573, 0.07315187435859002, 0.06892826480969184, 0.07830049356806958, 0.08384837355718774]
[0.07699759045509312, 0.08513131358234176, 0.07791098286114034, 0.08091338357340461, 0.07319757782441412, 0.07401784993600312, 0.07423507106744251, 0.08003899317295604, 0.08763444386148993]
[0.07975302164704061, 0.08718380924739351, 0.077606

In [12]:
groups_n =[]
group_sizes = ['2', '3', '4', '5', '6', '7', '8', '9', '10']
for i in range(len(group_sizes)):
    groups = np.loadtxt(os.path.join(group_dir_, group_sizes[i] + '.csv'), delimiter=',')
    groups = groups[int(len(groups)/5*4):int(len(groups)/5*5)]
    groups_n.append(groups)

NOVs = []
COVs = []
HitNOVs = []
HitCOVs = []
for lambda_ in tqdm(lambdas):
    top_n_lists_g = []
    top_n_items_g = []
    for groups in groups_n:

        group_size = len(groups[0])
        top_n_lists = []
        top_n_items = pd.DataFrame()

        for group in groups:
            rating_table = pd.DataFrame() 
            for member in group:
                prediction = svd[svd['uid'] == member].copy()
                if len(rating_table) == 0:
                    rating_table = prediction
                else:
                    rating_table = rating_table.set_index('tid').add(prediction.set_index('tid'), fill_value=0).reset_index()
            rating_table['rating'] /= group_size 
            rating_table['rating'] *= lambda_
            pop = df_pop.copy()
            pop['rating'] *= (1 - lambda_)
            rating_table = rating_table.set_index('tid').add(pop.set_index('tid'), fill_value=0).reset_index()
    #         rating_table = rating_table[rating_table['count'] == group_size]
            rating_table = rating_table.sort_values(by=['rating'],  ascending=False)
            rating_table = rating_table[:top_n_size]
            
            top_n_list = []
            for _, row in rating_table.iterrows():
                top_n_list.append(row[0])
            top_n_lists.append(top_n_list)
            
            if len(top_n_items) == 0:
                top_n_items = rating_table
            else:
                top_n_items = top_n_items.set_index('tid').add(rating_table.set_index('tid'), fill_value=0).reset_index()
                
        top_n_lists_g.append(top_n_lists)   
        top_n_items_g.append(top_n_items)
        
    COV = []    
    NOV = []
    HitNOV = []
    HitCOV = []
    for i in range(len(groups_n)):
        groups = groups_n[i]
        top_n_lists = top_n_lists_g[i]
        top_n_items = top_n_items_g[i]

        
        nov = 0
        hitNov = 0
        hits = np.zeros(len(df['tid'].unique()), dtype=bool)
        for j in (range(len(groups))):

            group = groups[j]
            top_n_list = top_n_lists[j]
            
            for k in range(len(group)):
                for l in range(top_n_size):
                    uid = group[k]
                    tid = top_n_list[l]
                    t = test[test['uid'] == uid]
                    t = t[t['tid'] == tid]
                    nov += (1-pop_dict[tid])
                    if len(t) > 0:
                        hits[int(tid)] = True
                        hitNov += (1-pop_dict[tid])
        NOV.append(nov/top_n_size/(len(groups)*len(groups[0])))
        COV.append(len(top_n_items)/len(df['tid'].unique()))
        HitNOV.append(hitNov/top_n_size/(len(groups)*len(groups[0])))
        HitCOV.append(list(hits).count(True)/len(df['tid'].unique()))
    NOVs.append(NOV)
    COVs.append(COV)
    HitNOVs.append(HitNOV)
    HitCOVs.append(HitCOV)
    
# for NOV in NOVs:
#     print(NOV)
# print()
for HitNOV in HitNOVs:
    print(HitNOV)
print()  
# for COV in COVs:
#     print(COV)
print()    
for HitCOV in HitCOVs:
    print(HitCOV)


[0.06960912906610714, 0.05351047860557307, 0.04744928296607205, 0.05701182867499765, 0.05719605785651095, 0.05278818767800933, 0.06242368596775731, 0.05590273729483243, 0.05716552990556141]
[0.08816030251068525, 0.05719001399090592, 0.04886411332633788, 0.06002003243346374, 0.058047857128160865, 0.05588367561085299, 0.0612557235524182, 0.055282862435713086, 0.05821812696747116]
[0.10989589998208489, 0.0598420922233881, 0.05031698146204966, 0.06286129924639891, 0.06252186079048622, 0.05617223804527059, 0.064092459219689, 0.05726879380254687, 0.05789808499475343]
[0.1318340030199882, 0.06484274804710265, 0.054128191675410955, 0.06447581799103304, 0.0649172890562311, 0.06231824314195773, 0.06946544405227506, 0.059960747736193684, 0.06112867261280169]
[0.14639105008573686, 0.06934060568963503, 0.059355981112277016, 0.06774444338452727, 0.06594397465177047, 0.06504754052722872, 0.07334672803586752, 0.06203541771922322, 0.062035020986358894]
[0.1535238527883703, 0.07073641716217789, 0.05877